In diesem Notebook wird die Pipeline aufgebaut.

In [1]:
%load_ext autoreload
%autoreload 2

import os
if os.getcwd() == '/home/jovyan/work': # jhub
    os.chdir("24FS_I4DS27/main/") 
    os.system("make reqs")
else: # local
    os.chdir("../")

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

from src.data.mri import MRIDataModule
from src.data.covidx import COVIDXDataModule
from src.utils.download import download_models
from src.utils.transform_perturbation import AddImagePerturbation
from src.utils.uap_helper import generate_adversarial_images_from_model_dataset, get_model
from src.models.imageclassifier import ImageClassifier

from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.figsize"] = (16, 8)

torch.set_float32_matmul_precision('high')

In [3]:
ENTITY = "24FS_I4DS27"
PROJECT = "baselines"
NUM_WORKERS = 8


def get_transform(perturbations: torch.Tensor = None, p: float = None, idx: int = None):
    if None in (perturbations, p, idx):
        return torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((224, 224), antialias=True),
            ]
        )
    return torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize((224, 224), antialias=True),
            AddImagePerturbation(perturbations, p, idx),
        ]
    )

def get_datamodule(dataset, transform=get_transform(), num_workers=0, batch_size=1, seed=42):
    if dataset == "covidx_data":
        return COVIDXDataModule(
            path="data/raw/COVIDX-CXR4",
            transform=transform,
            num_workers=num_workers,
            batch_size=batch_size,
            train_sample_size=0.05,
            train_shuffle=True,
            seed=seed,
        ).setup()

    elif dataset == "mri_data":
        return MRIDataModule(
            path="data/raw/Brain-Tumor-MRI",
            path_processed="data/processed/Brain-Tumor-MRI",
            transform=transform,
            num_workers=num_workers,
            batch_size=batch_size,
            train_shuffle=True,
            seed=seed,
        ).setup()

    else:
        raise ValueError("Invalid dataset")

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [4]:
# Download models if not present
models = download_models(ENTITY, PROJECT)

In [5]:
def pipeline(
    modelname: str,
    dataset: str,
    n_robustifications: int,
    i: int,
    n: int,
    t: int,
    p: int,
    lambda_norm: int,
    r: float,
    eps: float,
    lr_uap: float,
    seed: int,
    num_workers: int,
    device: str,
    verbose: bool = False,
):
    """
    Runs the pipeline for generating Universal Adversarial Perturbations (UAP) using the specified model and dataset.

    Parameters:
    modelname (str): The name of the model to be used for generating perturbations.
    dataset (str): The dataset to be used for training and evaluating the UAP.
    n_robustifications (int): The number of robustifications to apply.
    i (int): The number of UAPs to generate.
    n (int): The number of images to be used for generating UAPs.
    t (int): The number of retries to fool an image on the algorithm.
    p (int): The norm to be used for measuring perturbations (e.g., L2 norm, L∞ norm).
    lambda_norm (int): The regularization parameter for the norm.
    r (float): The desired fooling rate, which when achieved, saves the UAP.
    eps (float): A small positive constant for numerical stability in the loss function.
    lr_uap (float): The learning rate for the UAP optimization problem.
    seed (int): The random seed for reproducibility.
    num_workers (int): Number of worker threads for data loading.
    device (str): The device to be used for computation (e.g., 'cpu' or 'cuda').
    verbose (bool): Whether to print additional information during the pipeline.

    Returns:
    None
    """
    model, hparams = get_model(modelname=modelname, dataset=dataset, output_size=1, return_hparams=True)

    for current_robustification in range(n_robustifications):
        modelfolder = f"{modelname}-{dataset}-n_{n}-robustification_{current_robustification}"

        # check if the model has already been robustified and evaluated, if so, skip current iteration
        metrics_path_i = f"robustified_models/{modelfolder}/06_eval_robustified_test_uap_{i-1}/metrics.csv"
        metrics_path_next = f"robustified_models/{modelfolder}/06_eval_robustified_test_uap_{i}/metrics.csv"
        if os.path.exists(metrics_path_i) and not os.path.exists(metrics_path_next):
            print(f"Skipping robustification {current_robustification}, since already exists") 

            # load best model checkpoint
            model = ImageClassifier.load_from_checkpoint(
                checkpoint_path=f"robustified_models/{modelname}-{dataset}-n_{n}-robustification_{current_robustification}/04_robustify_model/model.ckpt",
                modelname=modelname,
                output_size=1,
                p_dropout_classifier=hparams["p_dropout_classifier"],
                lr=hparams["lr"],
                weight_decay=hparams["weight_decay"],
            )
            model.freeze()
            model.eval()
            
            continue

        # generate UAP
        loggerUAP = pl_loggers.CSVLogger(
            "robustified_models",
            name=modelfolder,
            version="01_UAPs_pre_robustification",
            flush_logs_every_n_steps=1,
        )
        perturbations = generate_adversarial_images_from_model_dataset(
            model,
            modelname,
            dataset,
            logger=loggerUAP,
            transform=get_transform(),
            i=i,
            n=n,
            r=r,
            p=p,
            lambda_norm=lambda_norm,
            t=t,
            eps=eps,
            lr_uap=lr_uap,
            seed=seed,
            num_workers=num_workers,
            device=device,
            verbose=verbose,
        ).cpu()
        perturbations = perturbations.detach()
        loggerUAP.save()

        # evaluate model on testdata
        loggerEvalUnrobustifiedTest = pl_loggers.CSVLogger(
            "robustified_models",
            name=modelfolder,
            version="02_eval_unrobustified_test",
            flush_logs_every_n_steps=1,
        )
        datamodule = get_datamodule(
            dataset=dataset,
            transform=get_transform(),
            num_workers=num_workers,
            batch_size=32,
            seed=seed,
        )
        trainer = Trainer(
            logger=loggerEvalUnrobustifiedTest,
        )
        trainer.test(model, datamodule.test_dataloader())

        # evaluate model on testdata + uaps
        for perturbation_idx in range(i):
            loggerEvalUnrobustifiedTestUAP = pl_loggers.CSVLogger(
                "robustified_models",
                name=modelfolder,
                version=f"03_eval_unrobustified_test_uap_{perturbation_idx}",
                flush_logs_every_n_steps=1,
            )
            datamodule = get_datamodule(
                dataset=dataset,
                transform=get_transform(perturbations, p=1, idx=perturbation_idx),
                num_workers=num_workers,
                batch_size=32,
                seed=seed,
            )
            trainer = Trainer(
                logger=loggerEvalUnrobustifiedTestUAP,
            )
            trainer.test(model, datamodule.test_dataloader())

        # unfreeze model
        model.unfreeze()
        model.train()

        # robustify model
        loggerRobustify = pl_loggers.CSVLogger(
            "robustified_models",
            name=modelfolder,
            version="04_robustify_model",
            flush_logs_every_n_steps=1,
        )
        datamodule = get_datamodule(
            dataset=dataset,
            transform=get_transform(perturbations, p=0.5),
            num_workers=num_workers,
            batch_size=32,
            seed=seed,
        )
        trainer = Trainer(
            max_epochs=50,
            log_every_n_steps=1,
            gradient_clip_val=0.5,
            accelerator="auto",
            logger=loggerRobustify,
            fast_dev_run=False,  # set to True to test run
            enable_progress_bar=True,
            enable_model_summary=True,
            callbacks=[
                EarlyStopping(monitor="val_loss", mode="min", patience=20),
                ModelCheckpoint(
                    monitor="val_loss",
                    mode="min",
                    save_top_k=1, 
                    save_last=False,  
                    dirpath=f"robustified_models/{modelname}-{dataset}-n_{n}-robustification_{current_robustification}/04_robustify_model",
                    filename="model",
                )
            ],
        )

        trainer.fit(model, datamodule.train_dataloader(), datamodule.val_dataloader())

        # load best model checkpoint
        model = ImageClassifier.load_from_checkpoint(
            checkpoint_path=f"robustified_models/{modelname}-{dataset}-n_{n}-robustification_{current_robustification}/04_robustify_model/model.ckpt",
            modelname=modelname,
            output_size=1,
            p_dropout_classifier=hparams["p_dropout_classifier"],
            lr=hparams["lr"],
            weight_decay=hparams["weight_decay"],
        )
        model.freeze()
        model.eval()

        # evaluate robustified model on testdata
        loggerEvalRobustifiedTest = pl_loggers.CSVLogger(
            "robustified_models",
            name=modelfolder,
            version="05_eval_robustified_test",
            flush_logs_every_n_steps=1,
        )
        datamodule = get_datamodule(
            dataset=dataset,
            transform=get_transform(perturbations, p=0.5),
            num_workers=num_workers,
            batch_size=32,
            seed=seed,
        )
        trainer = Trainer(
            logger=loggerEvalRobustifiedTest,
        )
        trainer.test(model, datamodule.test_dataloader())

        # evaluate robustified model on testdata + uaps
        for perturbation_idx in range(i):
            loggerEvalRobustifiedTestUAP = pl_loggers.CSVLogger(
                "robustified_models",
                name=modelfolder,
                version=f"06_eval_robustified_test_uap_{perturbation_idx}",
                flush_logs_every_n_steps=1,
            )
            datamodule = get_datamodule(
                dataset=dataset,
                transform=get_transform(perturbations, p=0.5, idx=perturbation_idx),
                num_workers=num_workers,
                batch_size=32,
                seed=seed,
            )
            trainer = Trainer(
                logger=loggerEvalRobustifiedTestUAP,
            )
            trainer.test(model, datamodule.test_dataloader())

In [6]:
modelname, dataset = "resnet50", "covidx_data"
print(f"\n---\nModel: {modelname} - Dataset: {dataset}")
print(f"Device: {device}")

max_retries = 5
for attempt in range(max_retries):
    try:
        pipeline(
            modelname=modelname,
            dataset=dataset,
            n_robustifications=5,
            i=5,
            n=100,
            t=20,
            p=2,
            lambda_norm=0.001,
            r=0.5,
            eps=1e-6,
            lr_uap=0.1,
            seed=42,
            num_workers=NUM_WORKERS,
            device=device,
            verbose=False,
        )
    # CUDA out of memory error
    except RuntimeError as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        if attempt >= max_retries - 1:
            print("All retries failed")


---
Model: resnet50 - Dataset: covidx_data
Device: cuda


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/jovyan/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 50.7MB/s]


Skipping robustification 0, since already exists
Skipping robustification 1, since already exists
Skipping robustification 2, since already exists


/opt/conda/lib/python3.11/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory robustified_models/resnet50-covidx_data-n_100-robustification_3/01_UAPs_pre_robustification exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Universal Pertubation:   0%|          | 0/5 [00:00<?, ?it/s]

...

Universal Pertubation:   0%|          | 0/5 [00:00<?, ?it/s]

Attempt 4 failed: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 
Skipping robustification 0, since already exists
Skipping robustification 1, since already exists
Skipping robustification 2, since already exists


/opt/conda/lib/python3.11/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory robustified_models/resnet50-covidx_data-n_100-robustification_3/01_UAPs_pre_robustification exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Universal Pertubation:   0%|          | 0/5 [00:00<?, ?it/s]

Attempt 5 failed: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 
All retries failed
